In [40]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb

from sklearn.model_selection import train_test_split, StratifiedKFold
#from sklearn.metrics import cohen_kappa_score, accuracy_score,balanced_accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import KFold # Use KFold for regression
from sklearn.metrics import mean_squared_error

#from plotly import express as px

#from utils import plot_confusion_matrix

import os

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

from joblib import load, dump


import gc


## Armado Dataset

In [41]:
# Paths
BASE_DIR = '../'
#PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train.csv")
#PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train.parquet")
PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train_producto.parquet")
PATH_TO_MODELS = os.path.join(BASE_DIR, "LGBM/models")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "LGBM/optuna_/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "LGBM/optuna_/optuna_artifacts")


SEED = 42
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATH_TO_TRAIN

'../Dataset/train_producto.parquet'

In [42]:
dataset = pd.read_parquet(PATH_TO_TRAIN)

In [43]:
dataset.shape

(29652, 273)

In [44]:
char_feats = dataset.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

In [45]:

float_cols = [f for f in dataset.columns if dataset[f].dtype == np.float16]

In [46]:
char_feats

['cat1', 'cat2', 'cat3', 'brand', 'estado_producto']

In [47]:
del dataset['producto_total_tn_y']

In [48]:
dataset = dataset.rename(columns={'producto_total_tn_x': 'tn'})


In [49]:
dataset['periodo']=dataset['periodo'].astype('Int64')
dataset['product_id']=dataset['product_id'].astype('Int64')
dataset['clientes_distintos']=dataset['clientes_distintos'].astype('Int64')
dataset['plan_precios_cuidados']=dataset['plan_precios_cuidados'].astype('Int64')
dataset['fin_vida_p']=dataset['fin_vida_p'].astype('Int64')


In [50]:
cols_a_convertir = [
    'cat_productos_distintos',
    'total_productos_distintos_lag1', 'total_productos_distintos_lag2', 'total_productos_distintos_lag3',
    'total_productos_distintos_lag4', 'total_productos_distintos_lag5', 'total_productos_distintos_lag6',
    'total_productos_distintos_lag7', 'total_productos_distintos_lag8', 'total_productos_distintos_lag9',
    'total_productos_distintos_lag10', 'total_productos_distintos_lag11', 'total_productos_distintos_lag12',
    'total_clientes_distintos_lag1', 'total_clientes_distintos_lag2', 'total_clientes_distintos_lag3',
    'total_clientes_distintos_lag4', 'total_clientes_distintos_lag5', 'total_clientes_distintos_lag6',
    'total_clientes_distintos_lag7', 'total_clientes_distintos_lag8', 'total_clientes_distintos_lag9',
    'total_clientes_distintos_lag10', 'total_clientes_distintos_lag11', 'total_clientes_distintos_lag12',
    'sku_size', 'cust_request_qty'
]

dataset[cols_a_convertir] = dataset[cols_a_convertir].astype('Int64')

In [51]:
gc.collect()

0

In [52]:
def reduce_mem_usage_old(df, verbose=True):
    """Reduce el uso de memoria de un DataFrame convirtiendo columnas al tipo más eficiente."""
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                # Siempre usar float32 para compatibilidad con LightGBM
                df[col] = df[col].astype(np.float32)

        else:
            # Convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(col_type):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')
        print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
        print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df


In [53]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        # Solo nos ocupamos de numéricos
        if not pd.api.types.is_numeric_dtype(col_type):
            if not pd.api.types.is_categorical_dtype(col_type):
                df[col] = df[col].astype('category')
            continue

        c_min, c_max = df[col].min(), df[col].max()
        has_na = df[col].isnull().any()

        # --- ENTEROS ---
        if pd.api.types.is_integer_dtype(col_type):
            # 1) Sin nulos → numpy ints
            if not has_na:
                if c_min >= np.iinfo(np.int8).min  and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)


        # --- FLOTANTES ---
        else:
            df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')
        print(f'Uso de memoria final del DataFrame:   {end_mem:.2f} MB')
        print(f'Memoria reducida en un {(100*(start_mem-end_mem)/start_mem):.2f}%')
    return df

In [54]:
#dataset = reduce_mem_usage(dataset)

In [55]:
gc.collect()

0

In [56]:
dataset.shape

(29652, 272)

In [57]:
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               ) #stratify = dataset.clase saco esto porque no es un problema de clasificacion

In [58]:
train.shape

(23721, 272)

In [59]:
# Eliminar el DataFrame
del dataset

gc.collect()

0

In [60]:
#train = reduce_mem_usage(train)
#gc.collect()

In [61]:
#test = reduce_mem_usage(test)
#gc.collect()

In [62]:
label = 'clase'
features = [col for col in train.columns if col != label]

In [63]:
train.to_parquet("train_tmp.parquet", index=False)
del train
gc.collect()

train = pd.read_parquet("train_tmp.parquet")


In [64]:
test.to_parquet("test_tmp.parquet", index=False)
del test
gc.collect()

test = pd.read_parquet("test_tmp.parquet")

In [65]:
train.shape

(23721, 272)

In [66]:
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [67]:
X_train.shape

(23721, 271)

In [68]:
del train
#del test

gc.collect()

0

In [69]:
print("X_train shape:", X_train.shape)

X_train shape: (23721, 271)


In [70]:
X_train = reduce_mem_usage(X_train)
gc.collect()


Uso de memoria inicial del DataFrame: 17.41 MB
Uso de memoria final del DataFrame:   26.34 MB
Memoria reducida en un -51.33%


C:\Users\diana\AppData\Local\Temp\ipykernel_15672\290016523.py:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):
C:\Users\diana\AppData\Local\Temp\ipykernel_15672\290016523.py:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):


0

In [71]:
#y_train = reduce_mem_usage(y_train)
gc.collect()

0

In [72]:
X_test = reduce_mem_usage(X_test)
gc.collect()

Uso de memoria inicial del DataFrame: 4.36 MB
Uso de memoria final del DataFrame:   6.60 MB
Memoria reducida en un -51.22%


C:\Users\diana\AppData\Local\Temp\ipykernel_15672\290016523.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int32)
C:\Users\diana\AppData\Local\Temp\ipykernel_15672\290016523.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.int16)
C:\Users\diana\AppData\Local\Temp\ipykernel_15672\290016523.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



0

In [73]:
#y_test = reduce_mem_usage(y_test)
#gc.collect()

## Entrenamiento

In [76]:
def lgb_objective1(trial):

    # Parámetros para LightGBM
    lgb_params = {
        'objective': 'regression',
        'metric': 'mse', # Cambiado a Mean Squared Error
        'verbosity': -1,
        # 'num_class': len(y_train.unique()), # Eliminar, esto es para clasificación
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    # Para regresión, scores_ensemble será un array 1D
    scores_ensemble = np.zeros(len(y_test),dtype=np.float32)

    # Score del 5 fold CV inicializado en 0
    score_folds = 0

    # Numero de splits del CV
    n_splits = 5

    # Objeto para hacer el split de CV (usar KFold para regresión, ya que StratifiedKFold es para clasificación)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED) # Added shuffle and random_state for reproducibility

    for i, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)): # Usar kf.split

        # Dataset in fold (donde entreno)
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                     label=y_train.iloc[if_index],
                                     free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                      label=y_train.iloc[oof_index],
                                      free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                              lgb_if_dataset,
                              valid_sets=lgb_oof_dataset,
                              callbacks=[lgb.early_stopping(50, verbose=False)],
                              # feval = mean_squared_error(y_test, preds, squared=False) # Eliminar o definir correctamente para custom metric
                             )

        # Acumulo las predicciones continuas para el conjunto de test
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)

        # Score del fold (registros de dataset train que en este fold quedan out of fold)
        # Calcular MSE para el fold OOF
        oof_preds = lgb_model.predict(X_train.iloc[oof_index])
        score_folds += mean_squared_error(y_train.iloc[oof_index], oof_preds) / n_splits
        
        # ⬇️ Liberar memoria de objetos pesados del fold
        del lgb_model, lgb_if_dataset, lgb_oof_dataset, oof_preds
        gc.collect()
        print(f"fin split {i}")

    # Promedio las predicciones del ensemble para el conjunto de test
    scores_ensemble = scores_ensemble / n_splits

    # Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    #predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    #predicted_df = test.copy()
    #predicted_df= pd.DataFrame({
    #'periodo': test['periodo'],
    #'customer_id': test['customer_id'],  # o el ID que necesites
    #'product_id' : test['product_id'],
    #'pred': scores_ensemble
    #})
    
    # Genero columna pred con predicciones promediadas de los 5 folds
    #predicted_df['pred'] = scores_ensemble
    # Grabo dataframe en temp_artifacts
    #dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    #upload_artifact(trial, predicted_filename, artifact_store)
    #del predicted_df
    gc.collect()
    
    # Grabo métricas de regresión en lugar de matriz de confusión
    # Puedes guardar métricas como MSE, RMSE, MAE, R2, etc. en un archivo de texto o log
    # Por ejemplo, calcular el MSE en el conjunto de test
    test_mse = mean_squared_error(y_test, scores_ensemble)
    trial.set_user_attr("test_mse", test_mse) # Almacenar MSE del test como atributo de usuario

    del scores_ensemble
    gc.collect()

    # Si quieres una visualización, podrías generar un scatter plot de predicciones vs valores reales
    # O un histograma de residuos. Aquí un placeholder para un "regression_metrics.txt"
    #regression_metrics_filename = os.path.join(PATH_TO_TEMP_FILES, f'regression_metrics_{trial.study.study_name}_{trial.number}.txt')
    #with open(regression_metrics_filename, 'w') as f:
    #    f.write(f"Test Mean Squared Error: {test_mse}\n")
    #    # Add other regression metrics if desired
    #upload_artifact(trial, regression_metrics_filename, artifact_store)
    
    gc.collect()

    # Devuelvo el score promedio de MSE del 5-fold CV a Optuna para que optimice en base a eso
    # Optuna minimiza por defecto, por lo que devolver MSE es apropiado.
    return score_folds

In [77]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='minimize',
                            storage="sqlite:///db.sqlite1",  # Specify the storage URL here.
                            study_name="Producto1",
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(lgb_objective, n_trials=2000, gc_after_trial=True)

[I 2025-06-22 21:41:19,550] A new study created in RDB with name: Producto1
C:\Users\diana\AppData\Local\Temp\ipykernel_15672\1368641655.py:73: FutureWarning: upload_artifact() got {'artifact_store', 'file_path', 'study_or_trial'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['study_or_trial', 'file_path', 'artifact_store'] in upload_artifact() have been deprecated since v4.0.0. They will be replaced with the corresponding keyword arguments in v6.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v4.0.0 for details.
  upload_artifact(trial, predicted_filename, artifact_store)
C:\Users\diana\AppData\Local\Temp\ipykernel_15672\1368641655.py:87: FutureWarning: upload_artifact() got {'artifact_store', 'file_path', 'study_or_trial'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['study_or_trial', 'file_path', 'artifact_store'] 

## Guardar modelos con mejores parametros

In [78]:
#Best trial
best_params = {
    'objective': 'regression',
    'metric': 'mse',
    'verbosity': -1,
    'lambda_l1': 8.276877432032801e-7,
    'lambda_l2': 1.205530599737045,
    'num_leaves': 176,
    'feature_fraction': 0.5138594072276339,
    'bagging_fraction': 0.9494316741835466,
    'bagging_freq': 6,
    'min_child_samples': 71,
}

In [ ]:
#Guarda 1 modelo como cada fold
def entrenar_lgbm_cv_y_guardar_modelos(best_params):

    os.makedirs(PATH_TO_MODELS, exist_ok=True)
    os.makedirs(PATH_TO_TEMP_FILES, exist_ok=True)

    # Inicializar ensemble de predicciones sobre X_test
    scores_ensemble = np.zeros(len(y_test))
    score_folds = 0
    n_splits = 5
    models = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    for fold, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)):
        print(f"\n🔁 Fold {fold+1}")

        # Dataset fold
        lgb_if_dataset = lgb.Dataset(
            data=X_train.iloc[if_index],
            label=y_train.iloc[if_index],
            free_raw_data=True, #cambié
            categorical_feature=char_feats
        )
        lgb_oof_dataset = lgb.Dataset(
            data=X_train.iloc[oof_index],
            label=y_train.iloc[oof_index],
            free_raw_data=True,#cambié
            categorical_feature=char_feats
        )

        # Entrenamiento
        lgb_model = lgb.train(
            best_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )

        # Guardar modelo
        model_path = os.path.join(PATH_TO_MODELS, f"lgb_model_fold{fold+1}.joblib")
        dump(lgb_model, model_path)
        print(f"✅ Modelo guardado en: {model_path}")
        models.append(lgb_model)

        # Predicciones test
        scores_ensemble += lgb_model.predict(X_test)

        # Evaluar OOF
        oof_preds = lgb_model.predict(X_train.iloc[oof_index])
        mse = mean_squared_error(y_train.iloc[oof_index], oof_preds)
        print(f"📉 MSE Fold {fold+1}: {mse:.4f}")
        score_folds += mse / n_splits

    # Promediar predicciones
    scores_ensemble /= n_splits

    # Guardar predicciones en test
    #predicted_df = X_test.copy() # reemplace por la seleccion de abajo
    predicted_df= pd.DataFrame({
    'periodo': test['periodo'],
    'customer_id': test['customer_id'],  # o el ID que necesites
    'product_id' : test['product_id'],
    'pred': scores_ensemble
    })

    predicted_df["true"] = y_test.values
    predicted_df["pred"] = scores_ensemble
    pred_path = os.path.join(PATH_TO_TEMP_FILES, "predicciones_test.joblib")
    dump(predicted_df, pred_path)
    print(f"\n✅ Predicciones guardadas en: {pred_path}")

    # Calcular y guardar métricas
    test_mse = mean_squared_error(y_test, scores_ensemble)
    metrics_path = os.path.join(PATH_TO_TEMP_FILES, "regression_metrics.txt")
    with open(metrics_path, "w") as f:
        f.write(f"Test Mean Squared Error (MSE): {test_mse:.6f}\n")
    print(f"📄 Métricas guardadas en: {metrics_path}")

    print(f"\n📊 MSE promedio en CV: {score_folds:.6f}")
    print(f"📉 MSE final en test: {test_mse:.6f}")

    return models, score_folds, test_mse


In [ ]:
#Ejecuta la funcion
models, mse_cv, mse_test = entrenar_lgbm_cv_y_guardar_modelos(best_params)




🔁 Fold 1
✅ Modelo guardado en: ../LGBM/models\lgb_model_fold1.joblib
📉 MSE Fold 1: 0.6531

🔁 Fold 2
✅ Modelo guardado en: ../LGBM/models\lgb_model_fold2.joblib
📉 MSE Fold 2: 0.7085

🔁 Fold 3
✅ Modelo guardado en: ../LGBM/models\lgb_model_fold3.joblib
📉 MSE Fold 3: 0.7949

🔁 Fold 4
✅ Modelo guardado en: ../LGBM/models\lgb_model_fold4.joblib
📉 MSE Fold 4: 0.7005

🔁 Fold 5
✅ Modelo guardado en: ../LGBM/models\lgb_model_fold5.joblib
📉 MSE Fold 5: 0.7783

✅ Predicciones guardadas en: ../LGBM/optuna/optuna_temp_artifacts\predicciones_test.joblib
📄 Métricas guardadas en: ../LGBM/optuna/optuna_temp_artifacts\regression_metrics.txt

📊 MSE promedio en CV: 0.727060
📉 MSE final en test: 0.721702


In [ ]:
#Guarda 1 solo modelo luego de evaluar con 5 fold
def entrenar_lgbm_final_con_cv_sinseed(best_params):
    os.makedirs(PATH_TO_MODELS, exist_ok=True)
    os.makedirs(PATH_TO_TEMP_FILES, exist_ok=True)

    # --- 1. Evaluación con CV ---
    score_folds = 0
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    for fold, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)):
        print(f"\n🔁 Fold {fold+1}")

        lgb_if_dataset = lgb.Dataset(
            data=X_train.iloc[if_index],
            label=y_train.iloc[if_index],
            free_raw_data=True,
            categorical_feature=char_feats
        )
        lgb_oof_dataset = lgb.Dataset(
            data=X_train.iloc[oof_index],
            label=y_train.iloc[oof_index],
            free_raw_data=True,
            categorical_feature=char_feats
        )

        model_cv = lgb.train(
            best_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )

        # Predicciones OOF para calcular MSE
        oof_preds = model_cv.predict(X_train.iloc[oof_index])
        mse = mean_squared_error(y_train.iloc[oof_index], oof_preds)
        print(f"📉 MSE Fold {fold+1}: {mse:.4f}")
        score_folds += mse / n_splits
        del lgb_if_dataset, lgb_oof_dataset, oof_preds
        gc.collect()
        
        
    print(f"\n📊 MSE promedio 5-Fold CV: {score_folds:.6f}")

    # --- 2. Entrenamiento final en todo el dataset ---
    print("\n🚀 Entrenando modelo final con todo el conjunto de entrenamiento...")

    final_dataset = lgb.Dataset(
        data=X_train,
        label=y_train,
        free_raw_data=True,
        categorical_feature=char_feats
    )

    final_model = lgb.train(
        best_params,
        final_dataset,
        num_boost_round=model_cv.best_iteration  # Usar mismo número de iteraciones que el promedio de CV
    )
    del final_dataset
    gc.collect
    
    # Guardar el modelo final
    model_path = os.path.join(PATH_TO_MODELS, "lgb_model_final.joblib")
    dump(final_model, model_path)
    print(f"✅ Modelo final guardado en: {model_path}")

    # --- 3. Evaluación en test ---
    y_pred_test = final_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    print(f"📉 MSE en test: {test_mse:.6f}")

    # Guardar predicciones
    #predicted_df = X_test.copy()
    #predicted_df["true"] = y_test.values
    #predicted_df["pred"] = y_pred_test
    #pred_path = os.path.join(PATH_TO_TEMP_FILES, "predicciones_test_final.joblib")
    #dump(predicted_df, pred_path)

    # Guardar métrica
    #metrics_path = os.path.join(PATH_TO_TEMP_FILES, "regression_metrics_final.txt")
    #with open(metrics_path, "w") as f:
    #    f.write(f"Test Mean Squared Error (MSE): {test_mse:.6f}\n")

    return final_model, score_folds, test_mse


In [90]:
#Guarda 1 solo modelo luego de evaluar con 5 fold
def entrenar_lgbm_final_con_cv(best_params,n):
    os.makedirs(PATH_TO_MODELS, exist_ok=True)
    os.makedirs(PATH_TO_TEMP_FILES, exist_ok=True)

    # --- 1. Evaluación con CV ---
    score_folds = 0
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    for fold, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)):
        print(f"\n🔁 Fold {fold+1}")

        lgb_if_dataset = lgb.Dataset(
            data=X_train.iloc[if_index],
            label=y_train.iloc[if_index],
            free_raw_data=True,
            categorical_feature=char_feats
        )
        lgb_oof_dataset = lgb.Dataset(
            data=X_train.iloc[oof_index],
            label=y_train.iloc[oof_index],
            free_raw_data=True,
            categorical_feature=char_feats
        )

        model_cv = lgb.train(
            best_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )

        # Predicciones OOF para calcular MSE
        oof_preds = model_cv.predict(X_train.iloc[oof_index])
        mse = mean_squared_error(y_train.iloc[oof_index], oof_preds)
        print(f"📉 MSE Fold {fold+1}: {mse:.4f}")
        score_folds += mse / n_splits
        del lgb_if_dataset, lgb_oof_dataset, oof_preds
        gc.collect()
        
        
    print(f"\n📊 MSE promedio 5-Fold CV: {score_folds:.6f}")

    # --- 2. Entrenamiento final en todo el dataset ---
    print("\n🚀 Entrenando modelo final con todo el conjunto de entrenamiento...")

    final_dataset = lgb.Dataset(
        data=X_train,
        label=y_train,
        free_raw_data=True,
        categorical_feature=char_feats
    )

    final_model = lgb.train(
        best_params,
        final_dataset,
        num_boost_round=model_cv.best_iteration  # Usar mismo número de iteraciones que el promedio de CV
    )
    del final_dataset
    gc.collect
    
    # Guardar el modelo final
    model_path = os.path.join(PATH_TO_MODELS, f"lgb_model_{n}.joblib")
    dump(final_model, model_path)
    print(f"✅ Modelo final guardado en: {model_path}")

    # --- 3. Evaluación en test ---
    y_pred_test = final_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    print(f"📉 MSE en test: {test_mse:.6f}")

    # Guardar predicciones
    #predicted_df = X_test.copy()
    #predicted_df["true"] = y_test.values
    #predicted_df["pred"] = y_pred_test
    #pred_path = os.path.join(PATH_TO_TEMP_FILES, "predicciones_test_final.joblib")
    #dump(predicted_df, pred_path)

    # Guardar métrica
    #metrics_path = os.path.join(PATH_TO_TEMP_FILES, "regression_metrics_final.txt")
    #with open(metrics_path, "w") as f:
    #    f.write(f"Test Mean Squared Error (MSE): {test_mse:.6f}\n")

    return final_model, score_folds, test_mse

In [94]:
SEEDS = [5,915,15,666,9999,37,45,125,90,1000,42]

n=50
for item in SEEDS:
    SEED = item
    # Función que da un solo modelo
    final_model, mse_cv, mse_test = entrenar_lgbm_final_con_cv(best_params,n)
    n=n+1


🔁 Fold 1
📉 MSE Fold 1: 997.4349

🔁 Fold 2
📉 MSE Fold 2: 1053.2180

🔁 Fold 3
📉 MSE Fold 3: 1061.6770

🔁 Fold 4
📉 MSE Fold 4: 977.3553

🔁 Fold 5
📉 MSE Fold 5: 758.2394

📊 MSE promedio 5-Fold CV: 969.584908

🚀 Entrenando modelo final con todo el conjunto de entrenamiento...
✅ Modelo final guardado en: ../LGBM/models\lgb_model_50.joblib
📉 MSE en test: 929.008101

🔁 Fold 1
📉 MSE Fold 1: 1409.3570

🔁 Fold 2
📉 MSE Fold 2: 1070.9001

🔁 Fold 3
📉 MSE Fold 3: 746.4554

🔁 Fold 4
📉 MSE Fold 4: 809.9685

🔁 Fold 5
📉 MSE Fold 5: 1078.0933

📊 MSE promedio 5-Fold CV: 1022.954870

🚀 Entrenando modelo final con todo el conjunto de entrenamiento...
✅ Modelo final guardado en: ../LGBM/models\lgb_model_51.joblib
📉 MSE en test: 920.913654

🔁 Fold 1
📉 MSE Fold 1: 722.8370

🔁 Fold 2
📉 MSE Fold 2: 932.7320

🔁 Fold 3
📉 MSE Fold 3: 956.9690

🔁 Fold 4
📉 MSE Fold 4: 878.5769

🔁 Fold 5
📉 MSE Fold 5: 1469.7532

📊 MSE promedio 5-Fold CV: 992.173600

🚀 Entrenando modelo final con todo el conjunto de entrenamiento...
✅ 